In [ ]:
import pandas as pd
import geopandas as gpd
from IPython.terminal.pt_inputhooks.osx import void_p
from shapely import wkt
from shapely.geometry import Point, Polygon, MultiPolygon, LineString
from scipy.spatial import cKDTree

import numpy as np
import ast


In [ ]:
cities_df = gpd.read_file('data/ads24-cities.csv')
communes_df = gpd.read_file('data/ads24-communes.csv')
countries_df = gpd.read_file('data/ads24-countries.csv')
powiats_df = gpd.read_file('data/ads24-powiats.csv')
voivodships_df = gpd.read_file('data/ads24-voivodships.csv')
roads_df = gpd.read_file('data/ads24-roads.csv')
railways_df = gpd.read_file('data/ads24-railways.csv')
buildings_df = gpd.read_file('data/ads24-buildings.csv')
trees_df = gpd.read_file('data/ads24-trees.csv')

In [ ]:
# cities_gdf = gpd.GeoDataFrame(cities_df, geometry='geometry', crs='EPSG:4326')
# communes_gdf = gpd.GeoDataFrame(communes_df, geometry='geometry', crs='EPSG:4326')
# countries_gdf = gpd.GeoDataFrame(countries_df, geometry='geometry', crs='EPSG:4326')
# powiats_gdf = gpd.GeoDataFrame(powiats_df, geometry='geometry', crs='EPSG:4326')
# voivodships_gdf = gpd.GeoDataFrame(voivodships_df, geometry='geometry', crs='EPSG:4326')
# roads_gdf = gpd.GeoDataFrame(roads_df, geometry='geometry', crs='EPSG:4326')
# railways_gdf = gpd.GeoDataFrame(railways_df, geometry='geometry', crs='EPSG:4326')
# buildings_gdf = gpd.GeoDataFrame(buildings_df, geometry='geometry', crs='EPSG:4326')
# trees_gdf = gpd.GeoDataFrame(trees_df, geometry='geometry', crs='EPSG:4326')

## 1. Cities which are within commune boundaries

In [ ]:
# cities_gdf

In [ ]:
# communes_gdf

In [ ]:
cities_df['geometry'] = cities_df['wkt'].apply(wkt.loads)
communes_df['geometry'] = communes_df['wkt'].apply(wkt.loads)

In [ ]:
cities_gdf = gpd.GeoDataFrame(cities_df, geometry='geometry', crs='EPSG:4326')
communes_gdf = gpd.GeoDataFrame(communes_df, geometry='geometry', crs='EPSG:4326')

In [ ]:
type(cities_gdf)

In [ ]:
#ennsure geometries are either Polygon or MultiPolygon
cities_gdf['geometry'] = cities_gdf['geometry'].apply(
    lambda geom: Polygon(list(geom.coords)) if isinstance(geom, LineString) else
    MultiPolygon([Polygon(list(line.coords)) for line in geom.geoms]) if isinstance(geom, MultiPolygon) else geom
)

communes_gdf['geometry'] = communes_gdf['geometry'].apply(
    lambda geom: Polygon(list(geom.coords)) if isinstance(geom, LineString) else
    MultiPolygon([Polygon(list(line.coords)) for line in geom.geoms]) if isinstance(geom, MultiPolygon) else geom
)

In [ ]:
cities_within_communes = gpd.sjoin(cities_gdf, communes_gdf, predicate='within')
cities_within_communes

In [ ]:
# # the correct sql query is CREATE EDGE within FROM (SELECT FROM region WHERE name = 'gmina Swarzędz' and type = "country") TO(SELECT FROM region WHERE name='Swarędz' and type='country')
#
# with open('city_within_commune.sql', 'w', encoding="utf-8") as f:
#     for index, row in cities_within_communes.iterrows():
#         f.write(f"CREATE EDGE within FROM (SELECT FROM region WHERE name = '{row['name_left']}' and type='city') TO(SELECT FROM region WHERE name='{row['name_right']}' and type='commune') UNIDIRECTIONAL;\n")

## 2. Communes which are within powiat boundaries

In [ ]:
powiats_df['geometry'] = powiats_df['wkt'].apply(wkt.loads)

In [ ]:
powiats_gdf = gpd.GeoDataFrame(powiats_df, geometry='geometry', crs='EPSG:4326')

In [ ]:
communes_gdf['geometry'] = communes_gdf['geometry'].apply(
    lambda geom: Polygon(list(geom.coords)) if isinstance(geom, LineString) else
    MultiPolygon([Polygon(list(line.coords)) for line in geom.geoms]) if isinstance(geom, MultiPolygon) else geom
)

powiats_gdf['geometry'] = powiats_gdf['geometry'].apply(
    lambda geom: Polygon(list(geom.coords)) if isinstance(geom, LineString) else
    MultiPolygon([Polygon(list(line.coords)) for line in geom.geoms]) if isinstance(geom, MultiPolygon) else geom
)

In [ ]:
communes_within_powiats = gpd.sjoin(communes_gdf, powiats_gdf, predicate='within')
communes_within_powiats

In [ ]:
# the correct sql query is CREATE EDGE within FROM (SELECT FROM region WHERE name = 'gmina Swarzędz' and type = "country") TO(SELECT FROM region WHERE name='Swarędz' and type='country')

with open('commune_within_powiat.sql', 'w', encoding="utf-8") as f:
    for index, row in communes_within_powiats.iterrows():
        f.write(f"CREATE EDGE within FROM (SELECT FROM region WHERE name = '{row['name_left']}' and type='commune') TO(SELECT FROM region WHERE name='{row['name_right']}' and type='powiat') UNIDIRECTIONAL;\n")

## 3. Powiats which are within voivodship boundaries

In [ ]:
voivodships_df['geometry'] = voivodships_df['wkt'].apply(wkt.loads)


In [ ]:
voivodships_gdf = gpd.GeoDataFrame(voivodships_df, geometry='geometry', crs='EPSG:4326')


In [ ]:
powiats_gdf['geometry'] = powiats_gdf['geometry'].apply(
    lambda geom: Polygon(list(geom.coords)) if isinstance(geom, LineString) else
    MultiPolygon([Polygon(list(line.coords)) for line in geom.geoms]) if isinstance(geom, MultiPolygon) else geom
)

voivodships_gdf['geometry'] = voivodships_gdf['geometry'].apply(
    lambda geom: Polygon(list(geom.coords)) if isinstance(geom, LineString) else
    MultiPolygon([Polygon(list(line.coords)) for line in geom.geoms]) if isinstance(geom, MultiPolygon) else geom
)

In [ ]:
powiats_within_voivodships = gpd.sjoin(powiats_gdf, voivodships_gdf, predicate='within')
powiats_within_voivodships


In [ ]:
# the correct sql query is CREATE EDGE within FROM (SELECT FROM region WHERE name = 'gmina Swarzędz' and type = "country") TO(SELECT FROM region WHERE name='Swarędz' and type='country')

with open('powiat_within_voivodship.sql', 'w', encoding="utf-8") as f:
    for index, row in powiats_within_voivodships.iterrows():
        f.write(f"CREATE EDGE within FROM (SELECT FROM region WHERE name = '{row['name_left']}' and type='powiat') TO(SELECT FROM region WHERE name='{row['name_right']}' and type='voivodship') UNIDIRECTIONAL;\n")

## 4. Voivodship which are within country boundaries

In [ ]:
countries_df['geometry'] = countries_df['wkt'].apply(wkt.loads)

In [ ]:
countries_gdf = gpd.GeoDataFrame(countries_df, geometry='geometry', crs='EPSG:4326')

In [ ]:
voivodships_gdf['geometry'] = voivodships_gdf['geometry'].apply(lambda geom: Polygon(list(geom.coords)) if isinstance(geom, LineString) else MultiPolygon([Polygon(list(line.coords)) for line in geom.geoms]))
countries_gdf['geometry'] = countries_gdf['geometry'].apply(lambda geom: Polygon(list(geom.coords)) if isinstance(geom, LineString) else MultiPolygon([Polygon(list(line.coords)) for line in geom.geoms]))



In [ ]:
voiv_within_countries = gpd.sjoin(voivodships_gdf, countries_gdf, predicate='within')
voiv_within_countries

In [ ]:
# the correct sql query is CREATE EDGE within FROM (SELECT FROM region WHERE name = 'gmina Swarzędz' and type = "country") TO(SELECT FROM region WHERE name='Swarędz' and type='country')

with open('voivodship_within_country.sql', 'w', encoding="utf-8") as f:
    for index, row in communes_within_powiats.iterrows():
        f.write(f"CREATE EDGE within FROM (SELECT FROM region WHERE name = '{row['name_left']}' and type='voivodship') TO(SELECT FROM region WHERE name='{row['name_right']}' and type='country') UNIDIRECTIONAL;\n")

## 5. Neighbouring (adjacent) communes

In [ ]:
communes_nei_communes = gpd.sjoin(communes_gdf, communes_gdf, predicate='intersects')
communes_nei_communes

In [ ]:
with open('commune_nei_commune.sql', 'w', encoding="utf-8") as f:
    for index, row in communes_nei_communes.iterrows():
        f.write(f"CREATE EDGE within FROM (SELECT FROM region WHERE name = '{row['name_left']}' and type='commune') TO(SELECT FROM region WHERE name='{row['name_right']}' and type='commune') UNIDIRECTIONAL;\n")

## 6. All neighbouring buildings not further than 500 meters apart; attributes: distance(meters)

In [ ]:
cities_within_communes = cities_within_communes[['name_left', 'name_right']]

In [ ]:
cities_within_communes.reset_index(drop=True, inplace=True)

In [ ]:
cities_within_communes

In [ ]:
# Filter out invalid WKT strings
valid_geometries = buildings_df['wkt'].str.contains(r'^(POINT|LINESTRING|POLYGON)', na=False)
if not valid_geometries.all():
    print("Warning: Some rows contain invalid WKT strings. These will be excluded.")
data = buildings_df[valid_geometries]

try:
    data['geometry'] = gpd.GeoSeries.from_wkt(data['wkt'])
except Exception as e:
    print("Error during WKT conversion:", e)
    exit()


In [ ]:
# Create GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry='geometry')

# Define CRS (assuming WGS84, EPSG:4326; change if necessary)
gdf.set_crs(epsg=4326, inplace=True)

# Project to a metric CRS for distance calculations (e.g., UTM or EPSG:3857)
gdf = gdf.to_crs(epsg=3857)

In [ ]:
# Find neighbors within 500 meters
neighbors = gdf.sindex.query_bulk(
    gdf.geometry,
    predicate='dwithin',
    distance=500,
    return_distance=True
)

In [ ]:
neighbor_data = {
    "building_1": neighbors[0],
    "building_2": neighbors[1],
    "distance_m": neighbors[2]
}

In [ ]:
neighbor_df = pd.DataFrame(neighbor_data)

neighbor_df['building_1_id'] = neighbor_df['building_1'].apply(lambda x: gdf.iloc[x]['id'])
neighbor_df['building_2_id'] = neighbor_df['building_2'].apply(lambda x: gdf.iloc[x]['id'])

## 7. All neighbouring trees not further than 50 meters apart; attributes: distance
(meters)

In [ ]:
trees_df['geometry'] = trees_df['wkt'].apply(wkt.loads)
trees_gdf = gpd.GeoDataFrame(trees_df, geometry='geometry', crs='EPSG:4326')

In [ ]:
trees_gdf = trees_gdf.to_crs(epsg=3857)
coords = trees_gdf.geometry.apply(lambda geom: (geom.x, geom.y)).to_list()

In [ ]:
tree = cKDTree(coords)
pairs = tree.query_pairs(r=50, output_type = 'ndarray')
print(len(pairs))

In [ ]:
df = gpd.GeoDataFrame({'id1': np.array((trees_gdf.iloc[pairs[:,0]]['id'])), 'id2': np.array((trees_gdf.iloc[pairs[:,1]]['id'])),'geo1': np.array((trees_gdf.iloc[pairs[:,0]]['geometry'])), 'geo2': np.array((trees_gdf.iloc[pairs[:,1]]['geometry']))})
ser1 = gpd.GeoSeries(np.array((trees_gdf.iloc[pairs[:,0]]['geometry'])))
ser2 = gpd.GeoSeries(np.array((trees_gdf.iloc[pairs[:,1]]['geometry'])))
df['distance'] = ser1.distance(ser2)

print(df['distance'])

In [ ]:
# f"CREATE EDGE neighbouring FROM (SELECT FROM region WHERE id = '{row['id_left']}' and type='tree') TO(SELECT FROM region WHERE id='{row['id_right']}' and type='tree') UNIDIRECTIONAL;\n"

## 8. Trees which are not further than 20 meters from a road

In [ ]:
trees_df['geometry'] = trees_df['wkt'].apply(wkt.loads)
roads_df['geometry'] = roads_df['wkt'].apply(wkt.loads)

In [ ]:
trees = gpd.GeoDataFrame(trees_df, geometry='geometry', crs='EPSG:4326')
roads = gpd.GeoDataFrame(roads_df, geometry='geometry', crs='EPSG:4326')

In [ ]:
trees = trees.to_crs("EPSG:3857")
roads = roads.to_crs("EPSG:3857")

In [ ]:
roads['buffer'] = roads.geometry.buffer(20)

In [ ]:
trees = trees.set_geometry('geometry')
roads = roads.set_geometry('buffer')

In [ ]:
trees_near_roads = gpd.sjoin(trees, roads[['id', 'buffer']], how='inner', predicate='within')

In [ ]:
trees_near_roads

In [ ]:
# f"CREATE EDGE neighbouring FROM (SELECT FROM region WHERE id = '{row['id_left']}' and type='road') TO(SELECT FROM region WHERE id='{row['id_right']}' and type='tree') UNIDIRECTIONAL;\n")

## 9. Roads which are connected through nodes; attributes: connecting node identifier,
which part of one road is connected to the other road (start, mid, end)

In [ ]:
roads_df['geometry'] = roads_df['wkt'].apply(wkt.loads)

gdf = gpd.GeoDataFrame(roads_df, geometry='geometry')

In [ ]:
gdf['coords'] = gdf['geometry'].apply(lambda geom: list(geom.coords))
exploded = gdf.explode('coords', ignore_index=True)
exploded['coords'] = exploded['coords'].apply(Point)

In [ ]:
exploded['coords'] = exploded['coords'].apply(lambda point: (point.x, point.y))

In [ ]:
node_counts = exploded.groupby('coords').size().reset_index(name='count')
shared_nodes = node_counts[node_counts['count'] > 1]['coords']

In [ ]:
exploded = exploded[exploded['coords'].isin(shared_nodes)]

In [ ]:
exploded['road_id'] = exploded.index // exploded['coords'].map(len)

In [ ]:
exploded['position'] = exploded.groupby('road_id').cumcount()

In [ ]:
exploded['prev_coords'] = exploded.groupby('road_id')['coords'].shift(1)
exploded['next_coords'] = exploded.groupby('road_id')['coords'].shift(-1)

relations = exploded[exploded['coords'].isin(shared_nodes)]
relations_df = relations[['road_id', 'coords', 'prev_coords', 'next_coords']]

In [ ]:
relations_df = relations_df.dropna(subset=['prev_coords', 'next_coords'], how='all')

def parse_coords(coords):
    if pd.isna(coords):
        return np.nan

    coords = str(coords)

    coords = coords.strip("()")
    return tuple(map(float, coords.split(", ")))

#apply the parsing function to 'coords', 'prev_coords', and 'next_coords'
relations_df['coords'] = relations_df['coords'].apply(parse_coords)
relations_df['prev_coords'] = relations_df['prev_coords'].apply(parse_coords)
relations_df['next_coords'] = relations_df['next_coords'].apply(parse_coords)

#group by 'road_id' and aggregate the data
agg_df = relations_df.groupby('road_id').agg({
    'coords': 'first',  # Keep the first coordinate for the group
    'prev_coords': lambda x: next((i for i in x if pd.notna(i)), np.nan),  # First non-null prev_coords
    'next_coords': lambda x: next((i for i in x if pd.notna(i)), np.nan)   # First non-null next_coords
}).reset_index()

print(agg_df)


In [ ]:
# f"CREATE EDGE intersects FROM (SELECT FROM region WHERE id = '{row['id_left']}' and type='road') TO(SELECT FROM region WHERE id='{row['id_right']}' and type='road') UNIDIRECTIONAL;\n")

## 10. Railways which cross roads; attributes: angle


In [ ]:
gdf_railways = gpd.GeoDataFrame(railways_df, geometry=gpd.GeoSeries.from_wkt(railways_df['wkt']))
gdf_roads = gpd.GeoDataFrame(roads_df, geometry=gpd.GeoSeries.from_wkt(roads_df['wkt']))

In [ ]:
gdf_railways.set_crs(epsg=4326, inplace=True)
gdf_roads.set_crs(epsg=4326, inplace=True)

In [ ]:
def get_segment_at_point(linestring, point, buffer_distance=1e-6):
    point_buffer = point.buffer(buffer_distance)

    intersection = linestring.intersection(point_buffer)

    if intersection.is_empty:
        return None

    if intersection.geom_type == 'MultiLineString':
        intersection = max(intersection.geoms, key=lambda x: x.length)

    if intersection.geom_type != 'LineString':
        return None

    return intersection

In [ ]:
def calculate_intersection_angle(railway_geom, road_geom):
    intersection_point = railway_geom.intersection(road_geom)
    if intersection_point.geom_type == 'MultiPoint':
        intersection_point = intersection_point.geoms[0]
    elif intersection_point.geom_type != 'Point':
        return None

    railway_segment = get_segment_at_point(railway_geom, intersection_point)
    road_segment = get_segment_at_point(road_geom, intersection_point)

    if railway_segment is None or road_segment is None:
        return None

    def get_vector(linestring):
        coords = list(linestring.coords)
        if len(coords) < 2:
            return None
        vector = np.array(coords[-1]) - np.array(coords[0])
        return vector / np.linalg.norm(vector)

    railway_vector = get_vector(railway_segment)
    road_vector = get_vector(road_segment)

    if railway_vector is None or road_vector is None:
        return None

    dot_product = np.clip(np.dot(railway_vector, road_vector), -1.0, 1.0)
    angle_rad = np.arccos(dot_product)
    angle_deg = np.degrees(angle_rad)

    return min(angle_deg, 180 - angle_deg)

In [ ]:
intersections = gpd.sjoin(gdf_railways, gdf_roads, how='inner', predicate='intersects')


In [ ]:
intersection_angles = []
for idx, row in intersections.iterrows():
    railway_geom = row['geometry']
    road_geom = gdf_roads.loc[row['index_right'], 'geometry']
    angle = calculate_intersection_angle(railway_geom, road_geom)
    intersection_angles.append(angle)

intersections['angle'] = intersection_angles

In [ ]:
result_df = intersections[['id_left', 'id_right', 'angle']].copy()
result_df = result_df.dropna()

angle_ranges = pd.cut(result_df['angle'],
                     bins=[0, 15, 30, 45, 60, 75, 90],
                     labels=['0-15°', '15-30°', '30-45°', '45-60°', '60-75°', '75-90°'])

In [ ]:
print("\nIntersections by angle range:")
print(angle_ranges.value_counts().sort_index())

In [ ]:
# f"CREATE EDGE intersects FROM (SELECT FROM region WHERE id = '{row['id_left']}' and type='road') TO(SELECT FROM region WHERE id='{row['id_right']}' and type='railway') UNIDIRECTIONAL;\n")